A parallel version of XOR using `neat.parallel`.

Since XOR is a simple experiment, a parallel version probably won't run any
faster than the single-process version, due to the overhead of
inter-process communication.

If your evaluation function is what's taking up most of your processing time
(and you should check by using a profiler while running single-process),
you should see a significant performance improvement by evaluating in parallel.

This example is only intended to show how to do a parallel experiment
in neat-python.  
You can of course roll your own parallelism mechanism
or inherit from `ParallelEvaluator` if you need to do something more complicated.


In [27]:
import math
import os
import time
import pickle

import neat
import pandas as pd
import numpy as np

import visualize

In [28]:
data = pd.read_csv('trainRW.csv')
data.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,Var11,Var12
0,8.0,0.180,0.37,0.9,0.049,36.0,109,0.99007,2.89,0.44,12.7,6
1,8.0,0.180,0.37,0.9,0.049,36.0,109,0.99007,2.89,0.44,12.7,6
2,7.3,0.650,0.00,1.2,0.065,15.0,21,0.99460,3.39,0.47,10.0,7
3,5.4,0.835,0.08,1.2,0.046,13.0,93,0.99240,3.57,0.85,13.0,7
4,8.7,0.820,0.02,1.2,0.070,36.0,48,0.99520,3.20,0.58,9.8,5


In [29]:
labels = data.loc[:,'Var12']
labels.head()

0    6
1    6
2    7
3    7
4    5
Name: Var12, dtype: int64

In [30]:
outputs = [ np.array([1 if x == idx-1 else 0 for x in range(10)]) for idx in labels]
predictors = data.loc[:, 'Var1':'Var11']
len(predictors.values[0])

11

In [31]:
def eval_genome(genome, config):
    """
    This function will be run in parallel by ParallelEvaluator.  It takes two
    arguments (a single genome and the genome class configuration data) and
    should return one float (that genome's fitness).
    Note that this function needs to be in module scope for multiprocessing.Pool
    (which is what ParallelEvaluator uses) to find it.  Because of this, make
    sure you check for __main__ before executing any code (as we do here in the
    last few lines in the file), otherwise you'll have made a fork bomb
    instead of a neuroevolution demo. :)
    """
    
    N_total = 350
    
    # Grab some random subset of test data
    test_data = data.sample(n=N_total)
    test_predictors = test_data.loc[:, 'Var1':'Var11'].values
    test_labels = test_data.loc[:,'Var12']
    test_outputs = [np.array([1 if x == idx-1 else 0 for x in range(10)]) for idx in test_labels]

    net = neat.nn.FeedForwardNetwork.create(genome, config)
    N_perfect = 0
    N_near = 0
    N_bad = 0
    for xi, xo in zip(test_predictors, test_outputs):

        outputs = net.activate(xi)
#         prediction = output.index(max(output))
        answer = xo.tolist().index(max(xo))
        for idx, output in enumerate(outputs):
            if idx == answer:
                N_perfect += output
            elif np.abs(idx - answer) == 1:
                N_near += output
            else:
                N_bad += output
            
#         error = abs(prediction - answer)
#         if error == 0:
#             N_perfect += 1
#         elif error == 1:
#             N_near += 1
#         else:
#             N_bad += 1
    
    
#     score = (2*N_perfect + N_near - 2*N_bad)/N_total
    score = (2*N_perfect + N_near - N_bad)/(N_total *10)

    return score


In [32]:
def run(config_file):
    # Load configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)

    # Run for up to 300 generations.
    pe = neat.ParallelEvaluator(4, eval_genome)
    winner = p.run(pe.evaluate, 300)
    
    with open('winner.pickle', 'wb') as f:
        pickle.dump(winner, f)

    # Display the winning genome.
    print('\nBest genome:\n{!s}'.format(winner))

    # Show output of the most fit genome against training data.
    print('\nOutput:')
    winner_net = neat.nn.FeedForwardNetwork.create(winner, config)
    
    # Use whole dataset for validation
    predictors = data.loc[:, 'Var1':'Var11'].values
    labels = data.loc[:,'Var12']
    outputs = [np.array([1 if x == idx-1 else 0 for x in range(10)]) for idx in labels]
    for xi, xo in zip(predictors, outputs):
        output = winner_net.activate(xi)
        print("input {!r}, expected output {!r}, got {!r}".format(xi, xo, output))

    #node_names = {-1:'A', -2: 'B', 0:'A XOR B'}
    visualize.draw_net(config, winner, True)
    visualize.plot_stats(stats, ylog=False, view=True)
    visualize.plot_species(stats, view=True)

In [33]:
# Determine path to configuration file. This path manipulation is
# here so that the script will run successfully regardless of the
# current working directory.
local_dir = os.getcwd()
config_path = os.path.join(local_dir, 'config-feedforward')

In [ ]:
run(config_path)


 ****** Running generation 0 ****** 

Population's average fitness: -0.15129 stdev: 0.13461
Best fitness: 0.21674 - size: (10, 110) - species 1 - id 55
Average adjusted fitness: 0.342
Mean genetic distance 1.101, standard deviation 0.127
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   150      0.2    0.342     0
Total extinctions: 0
Generation time: 8.457 sec

 ****** Running generation 1 ****** 

Population's average fitness: -0.02684 stdev: 0.11861
Best fitness: 0.22995 - size: (10, 110) - species 1 - id 55
Average adjusted fitness: 0.294
Mean genetic distance 1.216, standard deviation 0.139
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1   150      0.2    0.294     0
Total extinctions: 0
Generation time: 8.478 sec (8.467 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.07027 s

Population's average fitness: 0.23785 stdev: 0.05339
Best fitness: 0.28343 - size: (13, 101) - species 1 - id 2685
Average adjusted fitness: 0.319
Mean genetic distance 1.882, standard deviation 0.181
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   18   150      0.3    0.319     2
Total extinctions: 0
Generation time: 8.127 sec (8.202 average)

 ****** Running generation 19 ****** 

Population's average fitness: 0.21947 stdev: 0.05532
Best fitness: 0.28683 - size: (11, 91) - species 1 - id 2825
Average adjusted fitness: 0.176
Mean genetic distance 1.814, standard deviation 0.212
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   19   150      0.3    0.176     3
Total extinctions: 0
Generation time: 8.164 sec (8.196 average)

 ****** Running generation 20 ****** 

Population's average fitness: 0.21415 stdev: 0.05788
Best f

Population's average fitness: 0.21172 stdev: 0.07104
Best fitness: 0.28640 - size: (13, 89) - species 1 - id 5104
Average adjusted fitness: 0.296
Mean genetic distance 1.949, standard deviation 0.225
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   36   150      0.3    0.296     6
Total extinctions: 0
Generation time: 9.738 sec (9.092 average)

 ****** Running generation 37 ****** 

Population's average fitness: 0.20403 stdev: 0.06161
Best fitness: 0.28938 - size: (12, 86) - species 1 - id 5546
Average adjusted fitness: 0.263
Mean genetic distance 1.937, standard deviation 0.230
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   37   150      0.3    0.263     7
Total extinctions: 0
Generation time: 9.458 sec (9.127 average)

 ****** Running generation 38 ****** 

Population's average fitness: 0.21442 stdev: 0.06815
Best fi

Population's average fitness: 0.20202 stdev: 0.07898
Best fitness: 0.29372 - size: (13, 78) - species 1 - id 8058
Average adjusted fitness: 0.345
Mean genetic distance 2.109, standard deviation 0.260
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   54   150      0.3    0.345     9
Total extinctions: 0
Generation time: 7.043 sec (7.312 average)

 ****** Running generation 55 ****** 

Population's average fitness: 0.21142 stdev: 0.07345
Best fitness: 0.29827 - size: (13, 78) - species 1 - id 8058
Average adjusted fitness: 0.253
Mean genetic distance 2.071, standard deviation 0.218
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   55   150      0.3    0.253     0
Total extinctions: 0
Generation time: 8.081 sec (7.403 average)

 ****** Running generation 56 ****** 

Population's average fitness: 0.22193 stdev: 0.05553
Best fi

Population's average fitness: 0.23062 stdev: 0.05698
Best fitness: 0.29147 - size: (16, 75) - species 1 - id 10682
Average adjusted fitness: 0.165
Mean genetic distance 1.871, standard deviation 0.216
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   72   150      0.3    0.165     1
Total extinctions: 0
Generation time: 7.335 sec (7.869 average)

 ****** Running generation 73 ****** 

Population's average fitness: 0.23263 stdev: 0.05699
Best fitness: 0.29305 - size: (16, 74) - species 1 - id 10850
Average adjusted fitness: 0.193
Mean genetic distance 1.941, standard deviation 0.209
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   73   150      0.3    0.193     2
Total extinctions: 0
Generation time: 8.325 sec (7.952 average)

 ****** Running generation 74 ****** 

Population's average fitness: 0.21193 stdev: 0.06920
Best 

Population's average fitness: 0.24188 stdev: 0.05710
Best fitness: 0.29828 - size: (17, 70) - species 1 - id 13459
Average adjusted fitness: 0.210
Mean genetic distance 2.075, standard deviation 0.229
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   90   150      0.3    0.210     2
Total extinctions: 0
Generation time: 6.702 sec (6.778 average)

 ****** Running generation 91 ****** 

Population's average fitness: 0.23393 stdev: 0.06581
Best fitness: 0.29159 - size: (16, 65) - species 1 - id 13484
Average adjusted fitness: 0.292
Mean genetic distance 2.163, standard deviation 0.291
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   91   150      0.3    0.292     3
Total extinctions: 0
Generation time: 7.194 sec (6.833 average)

 ****** Running generation 92 ****** 

Population's average fitness: 0.23028 stdev: 0.06775
Best 

Population's average fitness: 0.21070 stdev: 0.07459
Best fitness: 0.28880 - size: (20, 62) - species 1 - id 16094
Average adjusted fitness: 0.234
Mean genetic distance 2.165, standard deviation 0.436
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1  108   150      0.3    0.234     8
Total extinctions: 0
Generation time: 6.857 sec (6.679 average)

 ****** Running generation 109 ****** 

Population's average fitness: 0.20909 stdev: 0.08774
Best fitness: 0.28728 - size: (20, 66) - species 1 - id 16161
Average adjusted fitness: 0.318
Mean genetic distance 2.172, standard deviation 0.497
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1  109   150      0.3    0.318     9
Total extinctions: 0
Generation time: 6.894 sec (6.723 average)

 ****** Running generation 110 ****** 

Population's average fitness: 0.21585 stdev: 0.07190
Bes

Population's average fitness: 0.23787 stdev: 0.06027
Best fitness: 0.29571 - size: (21, 63) - species 1 - id 18795
Average adjusted fitness: 0.213
Mean genetic distance 1.893, standard deviation 0.261
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1  126   150      0.3    0.213    26
Total extinctions: 0
Generation time: 7.780 sec (7.450 average)

 ****** Running generation 127 ****** 

Population's average fitness: 0.24018 stdev: 0.05575
Best fitness: 0.29084 - size: (24, 72) - species 1 - id 18835
Average adjusted fitness: 0.226
Mean genetic distance 1.884, standard deviation 0.219
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1  127   150      0.3    0.226    27
Total extinctions: 0
Generation time: 7.296 sec (7.393 average)

 ****** Running generation 128 ****** 

Population's average fitness: 0.23360 stdev: 0.06756
Bes

Population's average fitness: 0.25310 stdev: 0.06557
Best fitness: 0.29500 - size: (19, 53) - species 1 - id 21394
Average adjusted fitness: 0.487
Mean genetic distance 1.813, standard deviation 0.205
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1  144   150      0.3    0.487    11
Total extinctions: 0
Generation time: 6.688 sec (7.504 average)

 ****** Running generation 145 ****** 

Population's average fitness: 0.25207 stdev: 0.05592
Best fitness: 0.29299 - size: (22, 62) - species 1 - id 21560
Average adjusted fitness: 0.301
Mean genetic distance 1.897, standard deviation 0.176
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1  145   150      0.3    0.301    12
Total extinctions: 0
Generation time: 6.870 sec (7.435 average)

 ****** Running generation 146 ****** 

Population's average fitness: 0.24914 stdev: 0.06185
Bes

Population's average fitness: 0.23221 stdev: 0.07833
Best fitness: 0.29590 - size: (25, 63) - species 2 - id 23963
Average adjusted fitness: 0.351
